we are breaking up the pipeline

In [44]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install pyarrow
!pip install datasets
!pip install trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, max_length=250)
model = model.to("cuda")

what the pipeline was doing behind the curtain was tokenising the text, but we can just as easily do that in a separate step. Huggingface lets us initialize our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.


In [4]:
input_text = "My name is "

tokenizer(input_text, return_tensors="pt").to("cuda")

{'input_ids': tensor([[499, 564,  19,   3,   1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

the input ids are the ids of the tokens in the vocabulary, which the model then converts into the embeddings of the tokens. we can check this by decoding the ids back into words


In [5]:
tokenizer.decode([564])

'name'

part of the pipeline corresponds to the .generate() method, which takes the token ids and generates the next token ids. we can do this in a separate step as well

try with and without .to("cuda")

In [6]:
output = model.generate(tokenizer(input_text, return_tensors="pt").to("cuda")["input_ids"]).to("cuda")
output

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


tensor([[  0,   3,   9,   3,   7,   9, 967,   1]], device='cuda:0')

we then only need to decode the ids back into words to get the generated text

In [7]:
tokenizer.decode(output[0])

'<pad> a sailor</s>'

make a function that works like the pipeline, but with the tokenization and generation steps separated


In [3]:
def my_pipe(input_text, model):
    output = model.generate(tokenizer(input_text, return_tensors="pt").to("cuda")["input_ids"]).to("cuda")
    return tokenizer.decode(output[0])


## incstruction tuning

we will try to do machinet translation 

zero shot
one shot
few shot

finetune
and then do all again to see if they improved

the dataset iis the [OPUS-100](https://huggingface.co/datasets/Helsinki-NLP/opus-100) which contains translation pairs from over 100 languages. i chose the danish to english translation pairs because that makes it easier for me to evaluate the quality of the translations, so feel very free to choose a different language pair if you prefer. you can see the different language pairs available in the "Subset" part of the dataset viewer.


In [4]:
from datasets import load_dataset

ds = load_dataset("Helsinki-NLP/opus-100", "da-en")

In [10]:
ds

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

we'll use huggingface's datasets library to load the dataset. the dataset is already split into training, validation, and test sets, so we can use those directly. it's in a dict format, so we can just use the key to access the part of the data we need

In [ ]:
train = ds["train"]
train

Dataset({
    features: ['translation'],
    num_rows: 1000000
})

In [23]:
def unpack_cols(row):
    row["en"] = row["translation"]["en"]
    row["da"] = row["translation"]["da"]
    return row

train = train.map(unpack_cols, remove_columns=["translation"])
train

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Dataset({
    features: ['en', 'da'],
    num_rows: 1000000
})

In [24]:
train[250]

{'en': "- What'd she say?", 'da': '– Hvad sagde hun?'}

try to pick a few sentences and see how well the model can translate out of the box.

In [25]:
input_ids = tokenizer(train[250]['en'], return_tensors="pt").to("cuda")["input_ids"]
input_ids

tensor([[  3,  18, 363,  31,  26, 255, 497,  58,   1]], device='cuda:0')

In [26]:
tokenizer.decode(model.generate(input_ids).to("cuda")[0])

/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<pad> - "I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I\'m sorry, I'

In [27]:
input_ids = tokenizer(f"English: {train[250]['en']} Danish: ", return_tensors="pt").to("cuda")["input_ids"]
input_ids

tensor([[ 1566,    10,     3,    18,   363,    31,    26,   255,   497,    58,
         23124,    10,     3,     1]], device='cuda:0')

In [28]:
tokenizer.decode(model.generate(input_ids)[0])

'<pad> - What would she say?</s>'

In [29]:
input_ids = tokenizer(f"""
                      English: "What a wonderful day!"
                      Danish: "Sikke en vidunderlig dag!"

                      English: "How are you?"
                      Danish: "Hvordan har du det?"
                      
                      English: {train[250]['en']} 
                      Danish: """, 
                      return_tensors="pt").to("cuda")["input_ids"]
input_ids

tensor([[ 1566,    10,    96,  5680,     3,     9,  1627,   239,  4720, 23124,
            10,    96,   134,    23,  8511,    15,     3,    35,     3,  6961,
          7248,  2825,   836,   122,  4720,  1566,    10,    96,  7825,    33,
            25,  4609, 23124,    10,    96,   566,  1967,  3768,     3,  3272,
           146,    20,    17,  4609,  1566,    10,     3,    18,   363,    31,
            26,   255,   497,    58, 23124,    10,     3,     1]],
       device='cuda:0')

In [30]:
tokenizer.decode(model.generate(input_ids).to("cuda")[0])

'<pad> Danish: "What a wonderful day!"</s>'

now let's try instruction tuning the model to hopefully get a better result

the datasets library has a nice map method that we can use to apply a function to all the examples in the dataset. the map method can take a custom function, so we just need to write a function that prepares our data for the model.

write preprocessing function that takes in a row of the dataset
- defines an instruction
- appends the input text to the instruction
- creates a list of all input texts
- creates a new column in the dataset called "input_ids" that contains the token ids of the input text
- creates a list of all output texts
- creates a new column in the dataset called "labels" that contains the token ids of the output text
- returns the augmented row


In [33]:
def preprocessing_func(row):
    instruction = "Translate the following English sentence to Danish: "
    input_text = instruction + row['translation']['en']
    row["input_ids"] = tokenizer(input_text, padding="max_length", truncation=True, return_tensors="pt").to("cuda").input_ids[0] ## HOW TO EXPLAIN THIS?!
    target_text = row['translation']['da']
    row["labels"] = tokenizer(target_text, padding="max_length", truncation=True, return_tensors="pt").to("cuda").input_ids[0]
    return row

In [61]:
def preprocessing_func(batch):
    input_texts = ["English: " + row + " Danish: " for row in batch['en']]
    batch["input_ids"] = tokenizer(input_texts, padding="max_length", truncation=True, return_tensors="pt").to("cuda").input_ids
    target_texts = [row for row in batch['da']]
    batch["labels"] = tokenizer(target_texts, padding="max_length", truncation=True, return_tensors="pt").to("cuda").input_ids
    return batch

buckle up this will probably take a handful of minutes

CUT DATASET IN HALF?
DO VAL WITH MULTIPLE EPOCHS INSTEAD?

In [62]:
tokenized_train = train.map(preprocessing_func, batched=True)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [63]:
tokenized_train = tokenized_train.remove_columns(["en", "da"])
tokenized_train

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1000000
})

In [38]:
tokenized_train[0]

{'input_ids': [30355,
  15,
  8,
  826,
  1566,
  7142,
  12,
  23124,
  10,
  242,
  8,
  3,
  5080,
  188,
  16761,
  3201,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

We then want to initalize a Trainer class.

To do this, we have to defined the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional.

there are many things you can optimise here, like the learning rate, the batch size, the number of epochs, etc. but for now, we can just use the default values. if you want to change them, you can find the full list of arguments in the documentation.

In [64]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(output_dir="./flan-t5-small-da-en",
   per_device_train_batch_size=4,
   #learning_rate=1e-3,
   #weight_decay=0.01,
   max_steps=3000,
   predict_with_generate=True,
   push_to_hub=False,
   logging_steps=200,
)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   tokenizer=tokenizer,
)

/tmp/ipykernel_5816/4090253005.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


In [46]:
from trl import SFTConfig, SFTTrainer

training_args = SFTConfig(output_dir="/tmp")

trainer = SFTTrainer(
    model,
    train_dataset=train,
    args=training_args,
)

/home/ucloud/.local/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 512
  warnings.warn(


Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

KeyError: 'text'

In [ ]:
trainer.train()

Step,Training Loss
200,0.082300
400,0.085100
600,0.084300
800,0.091200
1000,0.128500
1200,0.140000
1400,0.132200


KeyboardInterrupt: 

In [33]:
trainer.save_model("instruct-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 250}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


In [34]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("instruct-model")

In [35]:
my_pipe("Translate the following English sentence to Danish: How are you?", instruct_model)

/opt/anaconda3/envs/nlp/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<pad> Wie s<unk>?</s>'